In [4]:
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!wget https://github.com/tesseract-ocr/tessdata_best/raw/main/tha.traineddata
!sudo mv tha.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
--2024-08-07 09:01:44--  https://github.com/tesseract-ocr/tessdata_best/raw/main/tha.traineddata
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/main/tha.traineddata [following]
--2024-08-07 09:01:44--  https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/main/tha.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 18

In [5]:
import pandas as pd
import os

import pytesseract
from PIL import Image
import requests
from io import BytesIO

import nltk
from nltk.translate.bleu_score import sentence_bleu

In [6]:
!unzip -q /content/OCR.zip

Streaming output truncated to the last 5000 lines.
  inflating: OCR/0401.jpg            
  inflating: OCR/0402.jpg            
  inflating: OCR/0403.jpg            
  inflating: OCR/0404.jpg            
  inflating: OCR/0405.jpg            
  inflating: OCR/0406.jpg            
  inflating: OCR/0407.jpg            
  inflating: OCR/0408.jpg            
  inflating: OCR/0409.jpg            
  inflating: OCR/0410.jpg            
  inflating: OCR/0411.jpg            
  inflating: OCR/0412.jpg            
  inflating: OCR/0413.jpg            
  inflating: OCR/0414.jpg            
  inflating: OCR/0415.jpg            
  inflating: OCR/0416.jpg            
  inflating: OCR/0417.jpg            
  inflating: OCR/0418.jpg            
  inflating: OCR/0419.jpg            
  inflating: OCR/0420.jpg            
  inflating: OCR/0421.jpg            
  inflating: OCR/0422.jpg            
  inflating: OCR/0423.jpg            
  inflating: OCR/0424.jpg            
  inflating: OCR/0425.jpg            

<hr>

# OCR

In [17]:
def ocr_image(image_path):
    text = pytesseract.image_to_string(Image.open(image_path), lang='eng')
    # text = text.replace("\n","")
    # text = text.replace("\\x0"," ")
    # text = text.replace("\\x0c"," ")
    # text = text.replace("   "," ")
    text = text.replace("\n\x0c"," ")
    text = text.replace(" ",'')
    return text
ocr_image("/content/OCR/0000.jpg")

'12:00:00'

In [18]:
image_dir = "/content/OCR"
ocr_predictions = {}
for i in range(1, 17):
    img_id = f"{i:04}.jpg"
    image_path = os.path.join(image_dir, img_id)
    ocr_predictions[img_id] = ocr_image(image_path)

In [19]:
ocr_predictions

{'0001.jpg': '12:00:01',
 '0002.jpg': '12:00:02',
 '0003.jpg': '12:00:03',
 '0004.jpg': '12:00:04',
 '0005.jpg': '12:00:05',
 '0006.jpg': '12:00:06',
 '0007.jpg': '12:00:07',
 '0008.jpg': '12:00:08',
 '0009.jpg': '12:00:09',
 '0010.jpg': '12:00:10',
 '0011.jpg': '42:00:11',
 '0012.jpg': '12:00:12',
 '0013.jpg': '12:00:13',
 '0014.jpg': '12:00:14',
 '0015.jpg': '12:00:15',
 '0016.jpg': '12:00:16'}

<hr>

# Bleu

In [ ]:
ocr_data = pd.read_csv("/content/v89OCR.csv")

In [ ]:
# ocr_predictions = {
#     '01.jpg': 'ดาวพระเคราะห์ ในวิชาเลข ๗ ตัว เรานำเอาดาวพระเคราะห์ ๗ ดวง ได้แก่ ดาวอาทิตย์ ดาวจันทร์ ดาวอังคาร ดาวพุธ ดาวพฤหัสบดี ดาวศุกร์ และดาวเสาร์เป็นหลักสำคัญในการพยากรณ์ แต่ก็ได้นำเอาความหมายของดาวสมมติ ซึ่งเป็นเพียงจุดเงาบนท้องฟ้าจากการคำนวณ ได่แก่ ดาวราหู และดาวเกตุมาร่วมพยากรณ์ด้วย โดยปรากฏอยู่ในตำแหน่งของฐานบวก หรือฐานกำลังพระเคราะห์ ตำนานต้นกำเนิดดาวพระเคราะห์ ก่อนที่จะกล่าวถึงรายการละเอียดเกี่ยวกับดาวพระเคราะห์ในความหมายด้านต่างๆ เพื่อให้เห็นภาพพจน์ เกิดความเข้าใจได้ง่าย ขอนำตำนานอันเป็นประวัติต้นกำเนิดของดาวพระเคราะห์มาเล่าขานไว้ก่อน ดังนี้ พระอาทิตย์ ตามตำนานโหราศาสตร์กล่าวเอาไว้ว่า พระอิศวรผู้เป็นเจ้าได้สร้างพระอาทิตย์ขึ้นมาด้วยการเอาราชสีห์ ๖ ตัว มาร่ายพระเวทให้เป็นผง แล้วห่อด้วยผ้าสีแดง จากนั้นทรงประพรมด้วยน้ำอมฤต จึงบังเกิดเป็นองค์พระสุริยาอาทิตย์เทพทินกร มีสีวรกายแดง ทรงทิพยอาภรณ์พรายแพรว แล้วไปด้วยแก้ปัทมราช วิมานที่ประทับก็สีแดงทรงสีหไกรสร (ราชสีห์) เป็นพาหนะสถิต ณ อิสานทิศ (ทิศตะวันออกเฉียงเหนือ) มีกำลัง ๖ (เท่ากับจำนวนราชสีห์ที่นำมาสร้าง) พระจันทร์ ตามตำนานโหราศาสตร์กล่าวเอาไว้ว่า พระอิศวรผู้เป็นเจ้า ได้สร้างพระจันทร์ขึ้นมาจากนางฟ้า ๑๕ นางนั้นละเอียดป่นเป็นผง แล้วทรงห่อด้วยผ้าสีขาว ประพรมด้วยน้ำอมฤต ก็บังเกิดเป็นพระจันทร์เทพบุตรขึ้นมา มีสีวรกายเป็นสีขาวนวล ทรงทิพยอาภรณ์ แก้วประพาฬพิจิตร มีวิมานที่สถิตเป็นสีแก้วมุกดา ทรงอัศวราช (ม้า) เป็นพาหนะ',
#     '02.jpg': 'สถิต ณ บูรพาทิศ (ทิศตะวันออก) มีกำลัง ๑๕ (เท่ากับจำนวนนางฟ้าที่นำมาสร้าง) พระอังคาร ตามตำนานโหราศาสตร์กล่าวเอาไว้ว่า พระอิศวรผู้เป็นเจ้าทรงสร้างพระอังคารขึ้นมาจากกระบือ ๘ ตัว โดยทรงร่ายพระเวทให้ร่างของกระบือทั้ง ๘ ป่นเป็นผง แล้วห่อด้วยผ้าสีสลัว (คือ สีของแก้วเพทาย) ทรงประพรมด้วยน้ำอมฤตเกิดเป็นองค์พระอังคารเทพเจ้าผู้เรืองฤทธิ์ขึ้น มีสีวรกายเป็นแก้วเพทาย ทรงรัตนโกเมนทร์เป็นอาภรณ์ มีวิมานที่ประทับเป็นสีทับทีม ทรงกาษรราช (กระบือ) เป็นพาหนะ สถิตในอาคเนย์ทิศ (ทิศตะวันออกเฉียงใต้) มีกำลัง ๘ (เท่ากับจำนวนกระบือที่นำมาสร้าง) พระพุธ ตามตำนานโหราศาสตร์กล่าวเอาไว้ว่า พระอิศวรผู้',
#     '03.jpg': 'พระศุกร์ ตามตำนานโหราศาสตร์กล่าวเอาไว้ว่า',
#     '04.jpg': 'พระจันทร์ (๒) เชื่อมโยงไปถึงรูปลักษณะ',
#     '05.jpg': 'เกี่ยวกับเพศ ได้แก่ เพศชาย เกี่ยวกับเชื้อชาติ',
#     '06.jpg': 'เกี่ยวกับสี ได้แก่ สีขาวนวล เกี่ยวกับรส',
#     '07.jpg': 'เกี่ยวกับอาหารการกิน ได้แก่',
#     '08.jpg': 'ความหมายทั่วไป',
#     '09.jpg': 'สถานที่ติดต่อในเรื่องความรัก กามารมณ์ ธนาคาร เกี่ยวกับทิศ ได้แก่ ทิศอุดร (เหนือ) เกี่ยวกับแร่ธาตุ ได้แก่ ทองแดง เกี่ยวกับสี ได้แก่ สีฟ้า หรือสีคราม เกี่ยวกับรส ได้แก่ รสฝาด เกี่ยวกับพืชผักผลไม้ ได้แก่ ส้มเกลี้ยง ส้มโอ ส้มเขียวหวาน ผลไม้ที่ผ่ากลางเป็นรัศมีทุกชนิด เกี่ยวกับอาหารการกิน ได้แก่ อาหารที่ผสมด้วยกะทิ ผสมด้วยนม เช่น แกงกะทิ ต้มข่าไก่ หรืออาหารสำเร็จรูป อาหารกระป๋อง ๗. ดาวเสาร์ ธาตุไฟ มีกำลัง ๑๐ ความหมายทั่วไป แสดงให้เห็นถึงความทุกข์ร้อน ครุ่นคิด ความตระหนี่ถี่เหนียว ความหวาดระแวง ความเห็นแก่ตัว ความอิจฉาริษยา ความอาฆาตพยาบาท ความดุดัน',
#     '10.jpg': 'พาลเกเร ความลี้ลับ ความผลัดพรากจากจร',
#     '11.jpg': 'ดาวพุธ (๔) ข้อเด่น : มีกิริยามารยาทงดงาม',
#     '12.jpg': 'ดาวเด่น – ดาวเสื่อม ดาวพระเคราะห์หลักที่ใช้ใน',
#     '13.jpg': 'เกี่ยวกับบุคคล ได้แก่ พวกที่ผิดกฎหมาย',
#     '14.jpg': 'เพื่อให้เป็นแนวทางในการพิจารณาประกอบการพยากรณ์',
#     '15.jpg': 'หมายเหตุ ๑. ดาวเด่น-ดาวเสีย ดังกล่าวมาแล้ว',
#     '16.jpg': 'ต่อมาท่านได้ลงเคราะห์ให้ดาวเกตุ',
# }

In [ ]:
def bleu_df(ocr_data,ocr_predict):
    bleu_scores = []
    for _,row in ocr_data.iterrows():
        img_id = row['img_id']                             #ดึง img id
        GT_text = row['text']                              #ดึง text
        Pd_text = ocr_predict.get(img_id, "")

        GT_tokens = [GT_text.split()]                      #ตัดคำ
        Pd_tokens = Pd_text.split()                        #ตัดคำ
        # print(GT_tokens)

        bleu_score = sentence_bleu(GT_tokens, Pd_tokens)
        bleu_scores.append((img_id, bleu_score))
        # print(bleu_scores)

        bleu_df = pd.DataFrame(bleu_scores, columns=['img_id', 'bleu_score'])
        bleu_df['bleu_score'] = bleu_df['bleu_score'].apply(lambda x: round(x,6))   # cast ค่า เอาแค่ 6 ตน มากกว่านี้เลยไม่สวย

    scores = [score for _, score in bleu_scores]
    average_bleu_score = sum(scores) / len(scores)
    return bleu_df,average_bleu_score


In [ ]:
df,lst = bleu_df(ocr_data,ocr_predictions)
print("Bleu now:",round(lst,5))
df

Bleu now: 0.05096


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

,img_id,bleu_score
0,01.jpg,0.000000
1,02.jpg,0.000000
2,03.jpg,0.038830
3,04.jpg,0.127836
4,05.jpg,0.067794
5,06.jpg,0.093605
6,07.jpg,0.146049
7,08.jpg,0.000000
8,09.jpg,0.080234
9,10.jpg,0.049665
